In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#!pip install -U bitsandbytes accelerate transformers trl peft

In [ ]:
# ✅ 1. Đăng nhập Hugging Face
from huggingface_hub import login
login(new_session=False)  #

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
import torch, math


In [ ]:
from datasets import load_dataset

# 1️⃣ Load dataset
dataset = load_dataset("hoangcung165/Metal-Health-Data")

# 2️⃣ (Tùy chọn) Lấy 1000 mẫu đầu để test
train_subset = dataset["train"].shuffle(seed=42).select(range(5000))

# 3️⃣ Hàm tiền xử lý theo format của Llama 3 Instruct
def preprocess_function(example):
    system_prompt = "You are a helpful and empathetic AI assistant supporting users in mental health conversations."

    # Bắt đầu bằng system prompt
    formatted_text = (
        "<|begin_of_text|>"
        "<|start_header_id|>system<|end_header_id|>\n"
        f"{system_prompt}<|eot_id|>"
    )

    # Lặp qua danh sách hội thoại trong mỗi example
    for turn in example["conversations"]:
        role = "user" if turn["from"] == "human" else "assistant"
        content = turn["value"].strip()
        formatted_text += (
            f"<|start_header_id|>{role}<|end_header_id|>\n"
            f"{content}<|eot_id|>"
        )

    # Kết thúc bằng token cuối hội thoại
    formatted_text += "<|end_of_text|>"

    return {"text": formatted_text}

# 4️⃣ Xử lý dataset
print("🔧 Đang xử lý dataset...")
processed_ds = train_subset.map(preprocess_function, remove_columns=["conversations"])

# 5️⃣ Hiển thị mẫu sau khi xử lý
print("✅ Mẫu sau khi xử lý:")
print(processed_ds[0]["text"][:1000])

# 6️⃣ Tách train/test
train_test_split = processed_ds.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

print(f"\nTrain samples: {len(train_dataset)}")
print(f"Eval samples: {len(eval_dataset)}")


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.truncation_side = "left"

model.config.use_cache = False


In [ ]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,                          # Rank của ma trận low-rank (8 là cân bằng RAM/hiệu quả)
    lora_alpha=16,                # Hệ số scaling
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,            # Dropout chống overfitting
    bias="none",
    task_type="CAUSAL_LM"
)


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # vì đây là causal LM
)


In [ ]:
from trl import SFTConfig

sft_config = SFTConfig(
    output_dir="./trained_model",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    warmup_ratio=0.03,
    optim="paged_adamw_32bit",
    lr_scheduler_type="constant",
    fp16=True,
    group_by_length=True,
    save_strategy="steps",
    logging_steps=50,
    max_length=1024,
    report_to="none",
)


In [ ]:
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset, 
    data_collator=data_collator,
    peft_config=lora_config,
)

In [ ]:
trainer.train()


In [ ]:
trainer.push_to_hub("Upload LoRA fine-tuned adapter")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

# 1. Thông tin mô hình
base_model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
lora_adapter_repo = "letri345/trained_model"  

# 2. Cấu hình quantization (dùng 4-bit để tiết kiệm RAM)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# 3. Load mô hình gốc
print("🔹 Đang tải mô hình gốc...")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

# 4. Load adapter LoRA đã fine-tuned
print("🔹 Đang tải adapter LoRA...")
model = PeftModel.from_pretrained(
    base_model,
    lora_adapter_repo,
    torch_dtype=torch.bfloat16
)

# ✅ Merge adapter vào model chính
print("🔧 Đang merge adapter vào mô hình gốc...")
model = model.merge_and_unload()

# 5. Load tokenizer
print("🔹 Đang tải tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.truncation_side = "left"

# ✅ Lưu mô hình đã merge
save_path = "llama3-8b-mental-health-merged"
print(f"💾 Đang lưu mô hình vào: {save_path}")
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("✅ Đã merge và lưu mô hình thành công.")
